# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python Packages
import pandas as pd
import numpy as np
import cassandra
import re
import os
import glob
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Print Working Directory 
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-27-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
 
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# Get total number of rows 
print(len(full_data_rows_list))
# See what the list of event data rows will look like
print(full_data_rows_list[:10])

8056
[['Stephen Lynch', 'Logged In', 'Jayden', 'M', '0', 'Bell', '182.85669', 'free', 'Dallas-Fort Worth-Arlington, TX', 'PUT', 'NextSong', '1.54099E+12', '829', "Jim Henson's Dead", '200', '1.54354E+12', '91'], ['Manowar', 'Logged In', 'Jacob', 'M', '0', 'Klein', '247.562', 'paid', 'Tampa-St. Petersburg-Clearwater, FL', 'PUT', 'NextSong', '1.54056E+12', '1049', 'Shell Shock', '200', '1.54354E+12', '73'], ['Morcheeba', 'Logged In', 'Jacob', 'M', '1', 'Klein', '257.41016', 'paid', 'Tampa-St. Petersburg-Clearwater, FL', 'PUT', 'NextSong', '1.54056E+12', '1049', 'Women Lose Weight (Feat: Slick Rick)', '200', '1.54354E+12', '73'], ['Maroon 5', 'Logged In', 'Jacob', 'M', '2', 'Klein', '231.23546', 'paid', 'Tampa-St. Petersburg-Clearwater, FL', 'PUT', 'NextSong', '1.54056E+12', '1049', "Won't Go Home Without You", '200', '1.54354E+12', '73'], ['Train', 'Logged In', 'Jacob', 'M', '3', 'Klein', '216.76363', 'paid', 'Tampa-St. Petersburg-Clearwater, FL', 'PUT', 'NextSong', '1.54056E+12', '1049'

In [4]:
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [6]:
# Print the first 5 rows in event_datafile_new.csv file
data_1=pd.read_csv('event_datafile_new.csv',nrows=5)
data_1

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Stephen Lynch,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",829,Jim Henson's Dead,91
1,Manowar,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Shell Shock,73
2,Morcheeba,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Women Lose Weight (Feat: Slick Rick),73
3,Maroon 5,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Won't Go Home Without You,73
4,Train,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Hey_ Soul Sister,73


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
# Create a Keyspace named keyspace_song. The keyspace will use one node for replication. 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS keyspace_song
    WITH REPLICATION=
    {'class':'SimpleStrategy','replication_factor':1}"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
# Create a KEYSPACE named keyspace_song 
try:
    session.set_keyspace('keyspace_song')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [10]:
# Query 1: we need to find the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
# we have to create a table song_length to contain all column in the table
# Primary key sessionId is the partition key and itemInSession is the clustering column.
query1="CREATE TABLE IF NOT EXISTS song_length"
query1=query1 + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"

try:
    session.execute(query1)
except Exception as e:
    print(e)

In [11]:
# We have provided part of the code to set up the CSV file. 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## create insert statment will insert sessionId, itemInSession, artist, song, length into the query1
        query1 = "INSERT INTO song_length (sessionId, itemInSession, artist, song, length)"
        query1 = query1 + "VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.`
        session.execute(query1, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

### Do a SELECT to verify that the data have been inserted into each table

In [20]:
# The SELECT statement will verify that the data was entered into the song_length table
query1="select * from song_length limit 5"
try:
    rows=session.execute(query1)
except Exception as e:
    print(e)
for row in rows:
    print(row.sessionid, row.iteminsession, row.artist, row.song, row.length)

23 0 Regina Spektor The Calculation (Album Version) 191.08526611328125
23 1 Octopus Project All Of The Champs That Ever Lived 250.95791625976562
23 2 Tegan And Sara So Jealous 180.06158447265625
23 3 Dragonette Okay Dolores 153.39056396484375
23 4 Lil Wayne / Eminem Drop The World 229.58975219726562


In [21]:
#The following query will give the artist,song title and song's length in the music app history list that was heard during
# sessionId = 338, and itemInSession = 4
query1="select * from song_length where sessionId=338 and itemInSession = 4"
try:
    rows=session.execute(query1)
except Exception as e:
    print(e)
for row in rows:
    print(row.artist,row.song,row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [22]:
## Query 2
# We need to find name of artist, song(sorted by itemInSession) and user(first and last name) for userid = 10, sessionid = 182.
# we create a table user_info_song, consisting of userId, sessionId, itemInSession, artist, song and username(fistname and lastname). 
# Our partition key is userId and sessionId. ItemInSession is the clustering column sorted in ascending order. 

query2="CREATE TABLE IF NOT EXISTS user_info_song"
query2=query2 + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, \
PRIMARY KEY ((userId, sessionId), itemInSession))"

try:
    session.execute(query2)
except Exception as e:
    print(e)

In [23]:
# We have provided part of the code to set up the CSV file.
file = 'event_datafile_new.csv'

# open the event_datafile_new.csv file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
# The insert statement will insert userId, sessionId, itemInSession, artist, song, and username into the query 2 variable for the user_info_song 
    for line in csvreader:
        query2 = "INSERT INTO user_info_song (userId, sessionId, artist, song, itemInSession, firstName, lastName)"
        query2 = query2 + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query2, (int(line[10]), int(line[8]), line[0], line[9], int(line[3]), line[1], line[4]))

In [24]:
# The SELECT statement will verify that the data was entered into the user_info_song table
query2="select * from user_info_song limit 5"
try:
    rows=session.execute(query2)
except Exception as e:
    print(e)
for row in rows:
    print(row.userid,row.sessionid,row.artist, row.song, row.iteminsession,row.firstname, row.lastname)

58 768 System of a Down Sad Statue 0 Emily Benson
58 768 Ghostland Observatory Stranger Lover 1 Emily Benson
58 768 Evergreen Terrace Zero 2 Emily Benson
85 776 Deftones Head Up (LP Version) 2 Kinsley Young
85 776 The Notorious B.I.G. Playa Hater (Amended Version) 3 Kinsley Young


In [25]:
# use the select statement to get the name of the artist,song(sorted by itemInSession) and user(first and last name) 
# for userid = 10, sessionid = 182
query2 = "SELECT * FROM user_info_song WHERE userId=10 AND sessionId=182"
rows = session.execute(query2)
for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [27]:
# Query 3: 
# We need to find every user name (first and last) in the music app history list who listened to the song 'All Hands Against His Own' 
# To do this we create table user_for_song with userid , song and username. 
# We use compound primary key of song and userId. Song is our partition key and userId is our clustering key.
query3="CREATE TABLE IF NOT EXISTS user_for_song"
query3=query3 + "(song text,userId int, firstName text, lastName text,PRIMARY KEY (song,userId))"

try:
    session.execute(query3)
except Exception as e:
    print(e)
      

In [28]:
# We have provided part of the code to set up the CSV file.
file = 'event_datafile_new.csv'

# open the event_datafile_new.csv file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# The INSERT statement will insert userId, song, and username into the `query3` variable  for the user_for_song table.
        query3 = "INSERT INTO user_for_song (song,userId, firstName, lastName)"
        query3 = query3 + "VALUES (%s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query3, ( line[9],int(line[10]),line[1], line[4]))

In [29]:
# The SELECT statement is used to verify that data was entered into the table user_for_song table
query3="select * from user_for_song limit 5"
try:
    rows=session.execute(query3)
except Exception as e:
    print(e)
for row in rows:
    print(row.song,row.userid,row.firstname, row.lastname)

Wonder What's Next 49 Chloe Cuevas
In The Dragon's Den 49 Chloe Cuevas
Too Tough (1994 Digital Remaster) 44 Aleena Kirby
Rio De Janeiro Blue (Album Version) 49 Chloe Cuevas
My Place 15 Lily Koch


In [30]:
# The query will give every user name (first and last) in the music app history who listened to the song 'All Hands Against His Own'
query3 = "SELECT * FROM user_for_song WHERE song='All Hands Against His Own'"
rows = session.execute(query3)
for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [31]:
## Drop the table before closing out the sessions

In [ ]:
session.execute('drop table song_length')
session.execute('drop table user_info_song')
session.execute('drop table user_info_song')

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()